In [1]:
import dolphindb as ddb
import time
import pandas as pd

# 读取dfs数据（仅读取数据）

In [2]:
start = time.perf_counter()

## 仅读表
script2 = """
// STATIC数据
STATIC = loadTable("dfs://A_STATIC",`pt)

// GUOKAI数据
// 读取表  注意和原表不同了，需要把TRADE_DT从日期变回string
FILLED_GUOKAI = loadTable('dfs://A_GUOKAI',`pt1)

// 读取BONDS数据
BONDS = loadTable("dfs://A_BONDS",`pt2)
"""

# 获取结束时间
end = time.perf_counter()
# 计算运行时间
runTime1 = end - start
runTime_ms1 = runTime1 * 1000
# 输出运行时间
print("运行时间：", runTime1, "秒")
print("运行时间：", runTime_ms1, "毫秒")


运行时间： 2.7700007194653153e-05 秒
运行时间： 0.027700007194653153 毫秒


# 读取dfs数据（读取+处理日期）

In [3]:
# time.clock()默认单位为s
# 获取开始时间
start = time.perf_counter()

In [4]:
# 连接服务器节点
s = ddb.session()
isConnect = s.connect("115.239.209.123", 8892, "admin", "123456")  # 连接数据库
print(isConnect)

True


In [5]:
# 读取表 + 处理日期数据

script = """
// STATIC数据
STATIC = loadTable("dfs://A_STATIC",`pt)

// GUOKAI数据
// 读取表  注意和原表不同了，需要把TRADE_DT从日期变回string
FILLED_GUOKAI = loadTable('dfs://A_GUOKAI',`pt1)

FILLED_GUOKAI = select * , strReplace(string(TRADE_DT), ".", "") from FILLED_GUOKAI
FILLED_GUOKAI.dropColumns!(`TRADE_DT)
FILLED_GUOKAI.rename!(`strReplace, `TRADE_DT)


// 读取BONDS数据
BONDS = loadTable("dfs://A_BONDS",`pt2)

BONDS = select * , strReplace(string(TRADE_DT), ".", "") from BONDS
BONDS.dropColumns!(`TRADE_DT)
BONDS.rename!(`strReplace, `TRADE_DT)
"""

s.run(script)

In [6]:
# 获取结束时间
end = time.perf_counter()
# 计算运行时间
runTime = end - start
runTime_ms = runTime * 1000
# 输出运行时间
print("运行时间：", runTime, "秒")
print("运行时间：", runTime_ms, "毫秒")


运行时间： 2.2116921999986516 秒
运行时间： 2211.6921999986516 毫秒


# 读取本地parquet文件

In [7]:
# time.clock()默认单位为s
# 获取开始时间
start = time.perf_counter()

In [8]:
# 读取本地数据
STATIC = pd.read_parquet('./Results/STATIC.parquet')     # 初始行数21930
BONDS = pd.read_parquet('./Results/BONDS.parquet')
FILLED_GUOKAI = pd.read_parquet('./Results/FILLED_GUOKAI.parquet')

In [9]:
# 获取结束时间
end = time.perf_counter()
# 计算运行时间
runTime = end - start
runTime_ms = runTime * 1000
# 输出运行时间
print("运行时间：", runTime, "秒")
print("运行时间：", runTime_ms, "毫秒")

运行时间： 3.0849808999919333 秒
运行时间： 3084.9808999919333 毫秒


# 读取服务器端parquet文件

In [10]:
# time.clock()默认单位为s
# 获取开始时间
start = time.perf_counter()

In [11]:
# 连接服务器节点
s = ddb.session()
isConnect = s.connect("192.168.100.43", 7988, "admin", "123456")  # 连接数据库
print(isConnect)

script = """

STATIC = parquet::loadParquet("/home/hanyang/luodata/STATIC.parquet")

BONDS = parquet::loadParquet("/home/hanyang/luodata/BONDS.parquet")

FILLED_GUOKAI = parquet::loadParquet("/home/hanyang/luodata/FILLED_GUOKAI.parquet")

"""

s.run(script)

True


In [12]:
# 获取结束时间
end = time.perf_counter()
# 计算运行时间
runTime = end - start
runTime_ms = runTime * 1000
# 输出运行时间
print("运行时间：", runTime, "秒")
print("运行时间：", runTime_ms, "毫秒")


运行时间： 21.042427299995325 秒
运行时间： 21042.427299995325 毫秒


# 结果汇总（某一次运行结果）

| 运行时间   | 读取dfs数据（仅读表） | 读取dfs数据（读表+处理日期） | 读取本地parquet数据 | 读取服务器端parquet数据 |
| --------- | --------------------- | -------------------------- | -------------------- | ---------------------- |
| 秒        | 2.7700007194653153e-05             | 2.2116                     | 3.0849                 | 21.0424                  |
| 毫秒      | 0.0277                 | 2211.6921                  | 3084.9808              | 21042.4272               |
